In [237]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [238]:
path_blast = 'data/wheat/selected_candidates.csv'
path_blast_filtered = 'data/wheat/selected_candidates.filtered.csv'


In [239]:
#TEs
params = {'min_len':50,'max_len':False,'min_distance':5,'max_q':1.1,'min_q':0.9,'min_pident':96,'min_qcov':94}


In [240]:
#read blast output
df = pd.read_csv(path_blast, sep='\t', header=None)
df.columns = ['qseqid','sseqid','qstart','qend','sstart','send','mismatch','gaps','pident','evalue','length','qlen','slen','qcovs']
print('initial:',len(df.index))
initial = len(df.index)

initial: 25371


In [241]:
#filter by length
if(params['min_len']):
    df = df[df.qlen > params['min_len']]
print('Min len: ' + str(len(df.index)))
min_length = str(len(df.index))

Min len: 25371


In [242]:
if(params['max_len']):
    df = df[df.qlen < params['max_len']]
print('Max len: ' + str(len(df.index)))    
max_length = str(len(df.index))

Max len: 25371


In [243]:
#filter by query / subject length treshold
df = df[((df.length / df.qlen) >= params['min_q'])]
print('min treshold:',len(df.index))
min_treshold = str(len(df.index))

min treshold: 15099


In [244]:
df = df[((df.length / df.qlen) <= params['max_q'])]
print('max treshold:',len(df.index))
max_treshold = str(len(df.index))

max treshold: 14373


In [246]:
#filter by qcov
df = df[(df.qcovs >= params['min_qcov'])]
print('Min qcov: ' + str(len(df.index)))
min_qcov = str(len(df.index))

Min qcov: 14373


In [247]:
df.head(1)

,qseqid,sseqid,qstart,qend,sstart,send,mismatch,gaps,pident,evalue,length,qlen,slen,qcovs
0,MITE_568_5B_263944566_263944833,5B,1,267,263944567,263944833,0,0,100.0,5.970000e-137,267,267,713149757,100


In [233]:
#MITE_476_4B_13282159_13282337
#DTT_Taes_Hades_BQ161682-1_6B_145574154_145574234
#DTT_Taes_Polyphemus_464G14-3_Un_40974812_40975048
#DTT_Taes_Icarus_BJ276129-1_5D_459108652_459108772
#DTT_Tdur_Athos_103H9-1_4A_621583700_621583776
#MITE_159_6D_443767375_443767541
#MITE_231_5A_445393104_445393226
#MITE_568_5B_263944566_263944833


In [249]:

#df_2 = df[df.qseqid=='DTT_Taes_Polyphemus_464G14-3_Un_40974812_40975048']res = []
#myrange = [i for i in range(100,20,-1)]
#for i in myrange:
    #filter by pident
#    df_new = df_2[(df_2.pident >= i)]
#    min_pident = str(len(df_new.index))
#    res.append(min_pident)
#df_plot = pd.DataFrame({'pid':myrange,'count':res})
#sns.scatterplot(x="pid", y="count", data=df_plot);


In [250]:
#filter by pident
df = df[(df.pident >= params['min_pident'])]
print('Min_pident: ' + str(len(df.index)))
min_pident = str(len(df.index))

Min_pident: 3083


In [251]:
#order sstart and send
df['new_sstart'] = df[['sstart','send']].min(axis=1)
df['new_ssend'] = df[['sstart','send']].max(axis=1)
df['sstart'] = df['new_sstart']
df['send'] = df['new_ssend']
df = df.drop('new_sstart',axis=1).drop('new_ssend',axis=1)
df = df.sort_values(by=['sseqid','sstart', 'send'])
df = df.reset_index(drop=True)
# sep by chr
dfs = {}
for seq in df.sseqid.unique():
    dfs[seq] = df[df.sseqid == seq]

In [252]:
df.head(3)

,qseqid,sseqid,qstart,qend,sstart,send,mismatch,gaps,pident,evalue,length,qlen,slen,qcovs
0,DTT_Taes_Hades_BQ161682-1_6B_145574154_145574234,1A,1,80,1209536,1209615,3,0,96.25,1.160000e-28,80,80,594102056,100
1,MITE_476_4B_13282159_13282337,1A,1,178,3675114,3675291,0,0,100.00,1.110000e-87,178,178,594102056,100
2,DTT_Taes_Polyphemus_464G14-3_Un_40974812_40975048,1A,1,236,8129530,8129765,8,0,96.61,1.950000e-106,236,236,594102056,100


In [253]:
# filter overlapped 
rows = []
discard = []
total = len(df.index)
count = 0
curr = 0
for index, row in df.iterrows():
    count += 1
    curr_new = int(count * 100 * 1.0 / (total * 1.0))
    if curr_new != curr:
        curr = curr_new
        if curr_new % 5 == 0:
            print(curr_new)
    if index in discard:
        continue
    for k2, v2 in df.loc[index:,].iterrows():
        if abs(v2.sstart - row.sstart) > params['min_distance']:
            break
        if abs(v2.sstart - row.sstart) <= params['min_distance'] and abs(v2.send - row.send) <= params['min_distance']:
            discard.append(k2)
    rows.append(row)


5
10
15
20
25
30
35
40
45
50
55
60
65
70
75
80
85
90
95
100


In [254]:
df = pd.DataFrame(rows)
print('Non overlapped: ' + str(len(df.index)))
non_overlapped = str(len(df.index))


Non overlapped: 2966


In [255]:
df.to_csv(path_blast_filtered, index=None, sep='\t')
path_blast_filtered

'data/wheat/selected_candidates.filtered.csv'

In [256]:
print('Initial: ' + str(initial))
print('Min len: ' + str(min_length))
print('Max len: ' + str(max_length))
print('Min treshold: ' + str(min_treshold))
print('Max treshold: ' + str(max_treshold))
print('Min pident: ' + str(min_pident))
print('Min qcov: ' + str(min_qcov))
print('Non overlapped: ' + str(non_overlapped))
print('Saved: ' + path_blast_filtered)

Initial: 25371
Min len: 25371
Max len: 25371
Min treshold: 15099
Max treshold: 14373
Min pident: 3083
Min qcov: 14373
Non overlapped: 2966
Saved: data/wheat/selected_candidates.filtered.csv


In [257]:
df.groupby('qseqid').agg(['count'])

,sseqid,qstart,qend,sstart,send,mismatch,gaps,pident,evalue,length,qlen,slen,qcovs
,count,count,count,count,count,count,count,count,count,count,count,count,count
qseqid,,,,,,,,,,,,,
DTT_Taes_Hades_BQ161682-1_6B_145574154_145574234,1251,1251,1251,1251,1251,1251,1251,1251,1251,1251,1251,1251,1251
DTT_Taes_Icarus_BJ276129-1_5D_459108652_459108772,72,72,72,72,72,72,72,72,72,72,72,72,72
DTT_Taes_Polyphemus_464G14-3_Un_40974812_40975048,78,78,78,78,78,78,78,78,78,78,78,78,78
DTT_Tdur_Athos_103H9-1_4A_621583700_621583776,378,378,378,378,378,378,378,378,378,378,378,378,378
MITE_159_6D_443767375_443767541,79,79,79,79,79,79,79,79,79,79,79,79,79
MITE_231_5A_445393104_445393226,53,53,53,53,53,53,53,53,53,53,53,53,53
MITE_476_4B_13282159_13282337,1032,1032,1032,1032,1032,1032,1032,1032,1032,1032,1032,1032,1032
MITE_568_5B_263944566_263944833,23,23,23,23,23,23,23,23,23,23,23,23,23
